Election de 2022 premier tour

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import missingno as msno
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

import os
os.listdir()

['migrations',
 'election2017.ipynb',
 'pop-2007.ipynb',
 '.DS_Store',
 'populations',
 'population_2007_2023.xlsx',
 'alembic.ini',
 'enums',
 'utils',
 'models',
 'election2012.ipynb',
 'exports',
 'labo1.ipynb',
 '.gitignore',
 'scripts',
 'db',
 'chomages',
 'labo2.ipynb',
 '.git',
 'main.py',
 'election2022.ipynb',
 'elections']

#### Importer la database

In [20]:
from db import SessionLocal
session = SessionLocal()

Import des datasets.

In [21]:
tour = 2
df = pd.read_excel(f'./elections/presidentielles-2022-{tour}.xlsx')


Visualise toutes les colones

In [22]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

df.head(1000)

,Code du département,Libellé du département,Etat saisie,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,% Blancs/Ins,% Blancs/Vot,Nuls,% Nuls/Ins,% Nuls/Vot,Exprimés,% Exp/Ins,% Exp/Vot,Sexe,Nom,Prénom,Voix,% Voix/Ins,% Voix/Exp,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
0,01,Ain,Complet,438250,109568,25.00,328682,75.00,19802,4.52,6.02,5875,1.34,1.79,303005,69.14,92.19,M,MACRON,Emmanuel,166635,38.02,54.99,F,LE PEN,Marine,136370,31.12,45.01
1,02,Aisne,Complet,373621,97965,26.22,275656,73.78,14465,3.87,5.25,5694,1.52,2.07,255497,68.38,92.69,M,MACRON,Emmanuel,102428,27.41,40.09,F,LE PEN,Marine,153069,40.97,59.91
2,03,Allier,Complet,249954,62311,24.93,187643,75.07,13866,5.55,7.39,6370,2.55,3.39,167407,66.98,89.22,M,MACRON,Emmanuel,87645,35.06,52.35,F,LE PEN,Marine,79762,31.91,47.65
3,04,Alpes-de-Haute-Provence,Complet,128077,31907,24.91,96170,75.09,7773,6.07,8.08,2599,2.03,2.70,85798,66.99,89.21,M,MACRON,Emmanuel,41659,32.53,48.55,F,LE PEN,Marine,44139,34.46,51.45
4,05,Hautes-Alpes,Complet,113540,27296,24.04,86244,75.96,7377,6.50,8.55,2208,1.94,2.56,76659,67.52,88.89,M,MACRON,Emmanuel,42213,37.18,55.07,F,LE PEN,Marine,34446,30.34,44.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,ZP,Polynésie française,Complet,205633,118714,57.73,86919,42.27,2398,1.17,2.76,1718,0.84,1.98,82803,40.27,95.26,M,MACRON,Emmanuel,42890,20.86,51.80,F,LE PEN,Marine,39913,19.41,48.20
103,ZS,Saint-Pierre-et-Miquelon,Complet,5043,2164,42.91,2879,57.09,219,4.34,7.61,139,2.76,4.83,2521,49.99,87.57,M,MACRON,Emmanuel,1243,24.65,49.31,F,LE PEN,Marine,1278,25.34,50.69
104,ZW,Wallis et Futuna,Complet,9514,3674,38.62,5840,61.38,100,1.05,1.71,60,0.63,1.03,5680,59.70,97.26,M,MACRON,Emmanuel,3831,40.27,67.45,F,LE PEN,Marine,1849,19.43,32.55
105,ZX,Saint-Martin/Saint-Barthélemy,Complet,24435,15107,61.83,9328,38.17,471,1.93,5.05,220,0.90,2.36,8637,35.35,92.59,M,MACRON,Emmanuel,3850,15.76,44.58,F,LE PEN,Marine,4787,19.59,55.42


Supprimer la colonne Etat saisie d’un DataFrame pandas.

In [23]:
df = df.drop(columns=["Etat saisie"], errors="ignore")

In [24]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

df.head(1000)

,Code du département,Libellé du département,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,% Blancs/Ins,% Blancs/Vot,Nuls,% Nuls/Ins,% Nuls/Vot,Exprimés,% Exp/Ins,% Exp/Vot,Sexe,Nom,Prénom,Voix,% Voix/Ins,% Voix/Exp,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
0,01,Ain,438250,109568,25.00,328682,75.00,19802,4.52,6.02,5875,1.34,1.79,303005,69.14,92.19,M,MACRON,Emmanuel,166635,38.02,54.99,F,LE PEN,Marine,136370,31.12,45.01
1,02,Aisne,373621,97965,26.22,275656,73.78,14465,3.87,5.25,5694,1.52,2.07,255497,68.38,92.69,M,MACRON,Emmanuel,102428,27.41,40.09,F,LE PEN,Marine,153069,40.97,59.91
2,03,Allier,249954,62311,24.93,187643,75.07,13866,5.55,7.39,6370,2.55,3.39,167407,66.98,89.22,M,MACRON,Emmanuel,87645,35.06,52.35,F,LE PEN,Marine,79762,31.91,47.65
3,04,Alpes-de-Haute-Provence,128077,31907,24.91,96170,75.09,7773,6.07,8.08,2599,2.03,2.70,85798,66.99,89.21,M,MACRON,Emmanuel,41659,32.53,48.55,F,LE PEN,Marine,44139,34.46,51.45
4,05,Hautes-Alpes,113540,27296,24.04,86244,75.96,7377,6.50,8.55,2208,1.94,2.56,76659,67.52,88.89,M,MACRON,Emmanuel,42213,37.18,55.07,F,LE PEN,Marine,34446,30.34,44.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,ZP,Polynésie française,205633,118714,57.73,86919,42.27,2398,1.17,2.76,1718,0.84,1.98,82803,40.27,95.26,M,MACRON,Emmanuel,42890,20.86,51.80,F,LE PEN,Marine,39913,19.41,48.20
103,ZS,Saint-Pierre-et-Miquelon,5043,2164,42.91,2879,57.09,219,4.34,7.61,139,2.76,4.83,2521,49.99,87.57,M,MACRON,Emmanuel,1243,24.65,49.31,F,LE PEN,Marine,1278,25.34,50.69
104,ZW,Wallis et Futuna,9514,3674,38.62,5840,61.38,100,1.05,1.71,60,0.63,1.03,5680,59.70,97.26,M,MACRON,Emmanuel,3831,40.27,67.45,F,LE PEN,Marine,1849,19.43,32.55
105,ZX,Saint-Martin/Saint-Barthélemy,24435,15107,61.83,9328,38.17,471,1.93,5.05,220,0.90,2.36,8637,35.35,92.59,M,MACRON,Emmanuel,3850,15.76,44.58,F,LE PEN,Marine,4787,19.59,55.42


In [25]:
df.shape

(107, 28)

Recuperer les information généraux sur les élections départemental

In [26]:
df_infos_general = df.iloc[:, :16]
df_infos_general.head()

,Code du département,Libellé du département,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,% Blancs/Ins,% Blancs/Vot,Nuls,% Nuls/Ins,% Nuls/Vot,Exprimés,% Exp/Ins,% Exp/Vot
0,01,Ain,438250,109568,25.00,328682,75.00,19802,4.52,6.02,5875,1.34,1.79,303005,69.14,92.19
1,02,Aisne,373621,97965,26.22,275656,73.78,14465,3.87,5.25,5694,1.52,2.07,255497,68.38,92.69
2,03,Allier,249954,62311,24.93,187643,75.07,13866,5.55,7.39,6370,2.55,3.39,167407,66.98,89.22
3,04,Alpes-de-Haute-Provence,128077,31907,24.91,96170,75.09,7773,6.07,8.08,2599,2.03,2.70,85798,66.99,89.21
4,05,Hautes-Alpes,113540,27296,24.04,86244,75.96,7377,6.50,8.55,2208,1.94,2.56,76659,67.52,88.89


Renommer les colonnes des information généraux sur les élections départemental

In [27]:
df_infos_general.columns = [
    "code_dept", "nom_dept", "nb_inscrits", "nb_abstentions",
    "pct_abstentions", "nb_votants", "pct_votants", "nb_blancs",
    "pct_blancs_inscrits", "pct_blancs_votants", "nb_nuls",
    "pct_nuls_inscrits", "pct_nuls_votants", "nb_exprimes",
    "pct_exprimes_inscrits", "pct_exprimes_votants"
]
df_infos_general.head()

,code_dept,nom_dept,nb_inscrits,nb_abstentions,pct_abstentions,nb_votants,pct_votants,nb_blancs,pct_blancs_inscrits,pct_blancs_votants,nb_nuls,pct_nuls_inscrits,pct_nuls_votants,nb_exprimes,pct_exprimes_inscrits,pct_exprimes_votants
0,01,Ain,438250,109568,25.00,328682,75.00,19802,4.52,6.02,5875,1.34,1.79,303005,69.14,92.19
1,02,Aisne,373621,97965,26.22,275656,73.78,14465,3.87,5.25,5694,1.52,2.07,255497,68.38,92.69
2,03,Allier,249954,62311,24.93,187643,75.07,13866,5.55,7.39,6370,2.55,3.39,167407,66.98,89.22
3,04,Alpes-de-Haute-Provence,128077,31907,24.91,96170,75.09,7773,6.07,8.08,2599,2.03,2.70,85798,66.99,89.21
4,05,Hautes-Alpes,113540,27296,24.04,86244,75.96,7377,6.50,8.55,2208,1.94,2.56,76659,67.52,88.89


In [28]:
df_infos_general.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   code_dept              107 non-null    object 
 1   nom_dept               107 non-null    object 
 2   nb_inscrits            107 non-null    int64  
 3   nb_abstentions         107 non-null    int64  
 4   pct_abstentions        107 non-null    float64
 5   nb_votants             107 non-null    int64  
 6   pct_votants            107 non-null    float64
 7   nb_blancs              107 non-null    int64  
 8   pct_blancs_inscrits    107 non-null    float64
 9   pct_blancs_votants     107 non-null    float64
 10  nb_nuls                107 non-null    int64  
 11  pct_nuls_inscrits      107 non-null    float64
 12  pct_nuls_votants       107 non-null    float64
 13  nb_exprimes            107 non-null    int64  
 14  pct_exprimes_inscrits  107 non-null    float64
 15  pct_ex

Suppression des données redondantes, risque de multicolinéarité en ML

In [29]:
colonnes_base = [
    "code_dept", "nom_dept",
    "nb_inscrits", "nb_abstentions", "nb_votants", 
    "nb_blancs", "nb_nuls"
]

df_infos_general_base = df_infos_general[colonnes_base]
df_infos_general_base.head()

,code_dept,nom_dept,nb_inscrits,nb_abstentions,nb_votants,nb_blancs,nb_nuls
0,01,Ain,438250,109568,328682,19802,5875
1,02,Aisne,373621,97965,275656,14465,5694
2,03,Allier,249954,62311,187643,13866,6370
3,04,Alpes-de-Haute-Provence,128077,31907,96170,7773,2599
4,05,Hautes-Alpes,113540,27296,86244,7377,2208


In [30]:
df_infos_general_base.shape

(107, 7)

Si les colonnes nb_blancs et nb_nuls existent, les fusionées.

In [31]:

if {"nb_blancs", "nb_nuls"}.issubset(df_infos_general_base.columns):
    df_infos_general_base["nb_blancs_nuls"] = df_infos_general_base["nb_blancs"].fillna(0) + df_infos_general_base["nb_nuls"].fillna(0)
    df_infos_general_base = df_infos_general_base.drop(columns=["nb_blancs", "nb_nuls"])

df_infos_general_base.head()

/tmp/ipykernel_1902/3627291289.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_infos_general_base["nb_blancs_nuls"] = df_infos_general_base["nb_blancs"].fillna(0) + df_infos_general_base["nb_nuls"].fillna(0)


,code_dept,nom_dept,nb_inscrits,nb_abstentions,nb_votants,nb_blancs_nuls
0,01,Ain,438250,109568,328682,25677
1,02,Aisne,373621,97965,275656,20159
2,03,Allier,249954,62311,187643,20236
3,04,Alpes-de-Haute-Provence,128077,31907,96170,10372
4,05,Hautes-Alpes,113540,27296,86244,9585


In [32]:
df_infos_general_base.shape

(107, 6)

In [33]:
df_departement = df_infos_general_base.iloc[:, :2]
df_departement.head()

,code_dept,nom_dept
0,01,Ain
1,02,Aisne
2,03,Allier
3,04,Alpes-de-Haute-Provence
4,05,Hautes-Alpes


In [34]:
df_stat_elections = df_infos_general_base.drop(columns=["nom_dept"])
df_stat_elections.head()

,code_dept,nb_inscrits,nb_abstentions,nb_votants,nb_blancs_nuls
0,01,438250,109568,328682,25677
1,02,373621,97965,275656,20159
2,03,249954,62311,187643,20236
3,04,128077,31907,96170,10372
4,05,113540,27296,86244,9585


Tratification pour consever les proportions

In [35]:
import pandas as pd

# Colonnes globales que l'on veut garder
cols_globales = ['Code du département', 'Libellé du département']

# Colonnes répétitives candidats
df_candidates = df.iloc[:, 16:]  # après % Exp/Vot
cols_per_candidate = 6
num_candidates = df_candidates.shape[1] // cols_per_candidate

dfs_list = []

for i in range(num_candidates):
    start_col = i * cols_per_candidate
    end_col = start_col + cols_per_candidate
    df_cand = df_candidates.iloc[:, start_col:end_col].copy()
    df_cand.columns = ['Sexe', 'Nom', 'Prenom', 'Voix', '% Voix/Ins', '% Voix/Exp']
    
    # Ajouter les informations de département
    df_cand = pd.concat([df[cols_globales], df_cand], axis=1)
    
    dfs_list.append(df_cand)

# Combiner tous les candidats
df_simple = pd.concat(dfs_list, ignore_index=True)

# Colonnes à garder
cols_to_keep = ['Code du département', 'Libellé du département', 'Sexe', 'Nom', 'Prenom', 'Voix']

# Nouveau dataframe simplifié
df_candidat_resultat = df_simple[cols_to_keep]

# Afficher toutes les lignes
pd.set_option('display.max_rows', 10)  # ou un nombre supérieur à 100
pd.set_option('display.max_columns', None)  # pour voir toutes les colonnes
pd.set_option('display.width', None)        # pour éviter la coupure à la largeur de l'écran


# Affichage des 5 premières lignes
df_candidat_resultat.head(1000)


,Code du département,Libellé du département,Sexe,Nom,Prenom,Voix
0,01,Ain,M,MACRON,Emmanuel,166635
1,02,Aisne,M,MACRON,Emmanuel,102428
2,03,Allier,M,MACRON,Emmanuel,87645
3,04,Alpes-de-Haute-Provence,M,MACRON,Emmanuel,41659
4,05,Hautes-Alpes,M,MACRON,Emmanuel,42213
...,...,...,...,...,...,...
209,ZP,Polynésie française,F,LE PEN,Marine,39913
210,ZS,Saint-Pierre-et-Miquelon,F,LE PEN,Marine,1278
211,ZW,Wallis et Futuna,F,LE PEN,Marine,1849
212,ZX,Saint-Martin/Saint-Barthélemy,F,LE PEN,Marine,4787


In [36]:
from datetime import date

from utils.election_importer import ElectionImporter
from enums.type_election import TypeElection

# =====================================================
# Instanciation
# =====================================================
importer = ElectionImporter()

# =====================================================
# Élection
# =====================================================
if tour == 1:
    election = importer.get_or_create_election(
        election_date=date(2022, 4, 10),  # 1er tour présidentielle 2022
        type_election=TypeElection.PRESIDENTIELLE,
        tour=tour
    )

elif tour == 2:
    election = importer.get_or_create_election(
        election_date=date(2022, 4, 24),  # 2nd tour présidentielle 2022
        type_election=TypeElection.PRESIDENTIELLE,
        tour=tour
    )

# =====================================================
# Départements
# =====================================================
#df_departement = edf.df_departement
if df_departement is not None and not df_departement.empty:
    importer.import_departements(df_departement)

# =====================================================
# Statistiques par département
# =====================================================
#df_stat_elections = edf.df_stat_elections
if df_stat_elections is not None and not df_stat_elections.empty:
    importer.import_stats(df_stat_elections, election.id)

# =====================================================
# Candidats & résultats
# =====================================================
#df_candidat_resultat = edf.df_candidat_resultat
if df_candidat_resultat is not None and not df_candidat_resultat.empty:
    importer.import_candidats_resultats(df_candidat_resultat, election.id)

print("✅ Import élection 2017 – Tour 1 terminé avec succès")


2026-01-04 07:42:46,195 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-01-04 07:42:46,201 INFO sqlalchemy.engine.Engine SELECT elections.id AS elections_id, elections.date AS elections_date, elections.type_election AS elections_type_election, elections.tour AS elections_tour 
FROM elections 
WHERE elections.date = %(date_1)s AND elections.type_election = %(type_election_1)s AND elections.tour = %(tour_1)s 
 LIMIT %(param_1)s
2026-01-04 07:42:46,203 INFO sqlalchemy.engine.Engine [cached since 10.57s ago] {'date_1': datetime.date(2022, 4, 24), 'type_election_1': 'PRESIDENTIELLE', 'tour_1': 2, 'param_1': 1}
2026-01-04 07:42:46,214 INFO sqlalchemy.engine.Engine INSERT INTO elections (date, type_election, tour) VALUES (%(date)s, %(type_election)s, %(tour)s) RETURNING elections.id
2026-01-04 07:42:46,216 INFO sqlalchemy.engine.Engine [cached since 10.57s ago] {'date': datetime.date(2022, 4, 24), 'type_election': 'PRESIDENTIELLE', 'tour': 2}
2026-01-04 07:42:46,220 INFO sqlalchemy.engine